# DQL

In [1]:
import requests
from pprint import pprint
import json 
import codecs


endpoint_diffbot = "http://kg.diffbot.com/kg/dql_endpoint"


def get_token():
    with open("../.diffbot-token", "r") as f:
        return f.read().strip()


def query_and_save(query, output_fpath):
    data = {
        "token": get_token(),
        "query": query,
        "type": "query"}
    r = requests.get(endpoint_diffbot, params=data)

    with codecs.open(output_fpath, "w", "utf-8") as out:   
        out.write(json.dumps(json.loads(r.content)))
    print(output_fpath)
    
    
entity_types = ["AdministrativeArea", "Article", "Corporation",
                    "DegreeEntity", "EducationMajorEntity", "EducationalInstitution",
                   "EmploymentCategory", "Image", "Intangible", "Landmark", "LocalBusiness",
                   "Miscellaneous", "Organization", "Person", "Place", "Product", "Role",
                   "Skill", "Video"]



In [10]:
query_and_save('type:Organization name:"Lombard Street" ', "data/ls.json")

data/ls.json


In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [15]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")

data/am.json
data/bo.json
data/ns.json
data/dm.json
